<a href="https://colab.research.google.com/github/Alepescinaa/ScientificTools/blob/main/Project1/Cp2/PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises: physics-informed neural network

Exercise on the implementation of physics-informed neural network.

Date: 2024

Course: 056936 - SCIENTIFIC COMPUTING TOOLS FOR ADVANCED MATHEMATICAL MODELLING (PAGANI STEFANO) [2023-24].

Example adapted from this [notebook](https://colab.research.google.com/drive/1qBrbgevkSBqqYc8bOPiaoJG1MBrBrluN?usp=share_link).


Let us consider the problem

\begin{aligned}
  & v_f *\sqrt(\nabla u\cdot D\nabla u) =1  \,, \quad (x,y) \in [-1.5,1.5] \times [-1.5,1.5]\,\\
\end{aligned}

where $\nu$ is unknown. We consider the PINN framework for solving the state/parameter estimation.

In [1]:
# import required libraries

import tensorflow as tf
import numpy as np
import scipy.io
from tensorflow import keras
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
!pip -q install pyDOE
from pyDOE import lhs  # for latin hypercube sampling

In [2]:
# set seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
!git clone https://github.com/Alepescinaa/ScientificTools
%cd ScientificTools/Project1/Cp2

fatal: destination path 'ScientificTools' already exists and is not an empty directory.
/content/ScientificTools/Project1/Cp2


In [4]:
# loading of the dataset

CP2data = np.load("CP2data.npz")
CP2data = CP2data['arr_0']

In [5]:
# loading of the estimate

CP2estimate = np.load("CP2estimate.npz")
CP2estimate = CP2estimate['arr_0']

In [6]:
CP2estimate[3]

array([-0.29207564,  5.27909383,  0.9323303 ])

# Solution

In [7]:
X, Y = np.meshgrid(np.linspace(-1.5,1.5,1501), np.linspace(-1.5,1.5,1501))
X_flat = tf.convert_to_tensor(np.hstack((X.flatten()[:,None],Y.flatten()[:,None])),dtype=tf.float64)
x0 = 1.5


In [8]:
def penalty(param, lower_bound, upper_bound):
    return tf.reduce_sum(tf.square(tf.maximum(param - upper_bound, 0)) +
                         tf.square(tf.maximum(lower_bound - param, 0)))
# PINN loss function
def loss(xcl,ycl,xmeas,ymeas,umeas,theta_fiber,a_ratio,y0):
    input_data=tf.concat([xmeas,ymeas],1)
    umeas_pred = PINN(input_data)
    r_pred   = r_PINN(xcl,ycl,theta_fiber,a_ratio)

    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.pow(r_pred,2))

    # bc
    param_2= -1.5+(1.5+1.5)*y0
    mse_bc= tf.pow( PINN( tf.transpose( tf.stack( [tf.constant([1.5],dtype=tf.float64), param_2] ) ) ) ,2)

    #penalty over param boundaries
    mse_penalty = penalty(theta_fiber,0,1)+penalty(a_ratio,0,1)+penalty(y0,0,1)

    #tf.print('mse_time',mse_meas)
    #tf.print('mse_param',mse_r)
    #tf.print('mse_bc',mse_bc)
    return mse_meas + 2*mse_r + 2*mse_bc + mse_penalty


# residual computation based on AD
@tf.function
def r_PINN(x,y,theta_fiber,a_ratio):
    input_data=tf.concat([x,y],1)
    u = PINN(input_data)
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    param_0=-np.pi/10+(np.pi/10+np.pi/10)*theta_fiber
    param_1=1+(9-1)*a_ratio
    theta0 = pi/2 - param_0
    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    #D = ((1/param[1])* tf.linalg.matmul(a,tf.transpose(a)) + tf.linalg.matmul(b,tf.transpose(b)))
    #return  tf.linalg.matmul(tf.transpose(u_grad), tf.linalg.matmul(D,u_grad)) - 1/100**2
    D_00 = 1 / param_1 * a[0]**2 + b[0]**2
    D_01 = 1 / param_1 * a[0] * a[1] + b[0] * b[1]
    D_10 = 1 / param_1 * a[0] * a[1] + b[0] * b[1]
    D_11 = 1 / param_1 * a[1]**2 + b[1]**2

    return tf.sqrt((u_x * D_00 * u_x + u_x * D_01 * u_y + u_y * D_10 * u_x + u_y * D_11 * u_y))  - 1/100



# neural network weight gradients
@tf.function
def grad(model,xcl,ycl,xmeas,ymeas,umeas,theta_fiber,a_ratio,y0):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(xcl,ycl,xmeas,ymeas,umeas,theta_fiber,a_ratio,y0)
        grads = tape.gradient(loss_value,model.trainable_variables)
        grad_tf = tape.gradient(loss_value,theta_fiber)
        grad_ar = tape.gradient(loss_value,a_ratio)
        grad_y0 = tape.gradient(loss_value,y0)
    return loss_value, grads, grad_tf, grad_ar, grad_y0

In [9]:
from sklearn.model_selection import train_test_split
# collocation points
Ncl = 20000
Xcl = lhs(2,Ncl)
xcl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,0],dtype=tf.float64),axis=-1)
ycl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,1],dtype=tf.float64),axis=-1)
X_coll = tf.concat([xcl,ycl],1)

# measurement points
ind_disp = 3

xmeas = CP2data[ind_disp][0]
ymeas = CP2data[ind_disp][1]
tmeas = CP2data[ind_disp][2]
xmeas_train, xmeas_val, ymeas_train, ymeas_val, tmeas_train, tmeas_val = train_test_split(xmeas, ymeas, tmeas, test_size=0.1)
xmeas_train = tf.constant(xmeas_train.reshape(18, 1), dtype=tf.float64)
ymeas_train = tf.constant(ymeas_train.reshape(18, 1), dtype=tf.float64)
tmeas_train = tf.constant(tmeas_train.reshape(18, 1), dtype=tf.float64)
xmeas_val = tf.constant(xmeas_val.reshape(2, 1), dtype=tf.float64)
ymeas_val = tf.constant(ymeas_val.reshape(2, 1), dtype=tf.float64)
tmeas_val = tf.constant(tmeas_val.reshape(2, 1), dtype=tf.float64)

In [10]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM


regularization_strength = 1e-3

PINN = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,),
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
     tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Reshape((1, 64)),

    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 64)),

    tf.keras.layers.Dense(128, activation='relu',
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
     tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(32, activation='relu',
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
     tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(1, activation=None,
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64)
])


In [11]:
from scipy.interpolate import RBFInterpolator

def checkpoint1_solution(x, y, t, X, Y, s_value=0.05, s_aniso_1=0.5, s_aniso_2=0.5):
    coordinates = np.column_stack((x, y))

    mesh_coordinates=np.column_stack((X.ravel(), Y.ravel()))

    s = [s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_aniso_1, s_value,s_value,s_value,s_value, s_aniso_2,s_value,s_value,s_value,s_value]

    rbf = RBFInterpolator(coordinates, t, neighbors=None, smoothing=s, kernel='thin_plate_spline', epsilon=None, degree=1)

    time_pred = rbf(mesh_coordinates)
    time_pred=time_pred.reshape(1501,1501)

    return time_pred

In [12]:
time_pred=checkpoint1_solution(xmeas, ymeas, tmeas, X, Y, s_value=0.05, s_aniso_1=0.5, s_aniso_2=0.5)

In [13]:
y0_initial=Y[np.where(time_pred==np.min(time_pred))]
y0_initial_scaled=(y0_initial+1.5)/3

In [ ]:


#param 0 -> fiber angle (-pi/10,pi/10)
#param 1-> aniso (0,1)
#param 2-> source y0 (-1.5,1,5)

pi = tf.constant(np.pi,dtype=tf.float64)
theta_fiber = tf.Variable([0.5], trainable=True,dtype=tf.float64)
a_ratio = tf.Variable([0.5], trainable=True,dtype=tf.float64)
y0 = tf.Variable([y0_initial_scaled[0]], trainable=True,dtype=tf.float64)


param0_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
param0_optimizer.build(PINN.trainable_variables + [theta_fiber])

param1_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
param1_optimizer.build(PINN.trainable_variables + [a_ratio])

param2_optimizer = tf.keras.optimizers.Adam(learning_rate=0.008,beta_1=0.99)
param2_optimizer.build(PINN.trainable_variables + [y0])

"""
initial_learning_rate = 0.01
tf_optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate,beta_1=0.99)
tf_optimizer.build(PINN.trainable_variables + [theta_fiber,a_ratio,y0])


for iter in range(800):


  loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

  param0_optimizer.apply_gradients(zip(grads,PINN.trainable_variables))

  loss_value_val, _, _,_,_ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)

  if ((iter+1) % 100 == 0):
    print('iter =  '+str(iter+1))
    #loss_value_np=loss_value.numpy()
    #print('loss = {:.4f}'.format(loss_value_np ))
    tf.print('loss =' , loss_value)
    tf.print('loss_val_param =' , loss_value_val)

    theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
    a_ratio_res=1+8*a_ratio
    y0_res=-1.5+3*y0
    print(theta_fiber_res.numpy())
    print(a_ratio_res.numpy())
    print(y0_res.numpy())
"""

for iter in range(1000):


  loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

  param0_optimizer.apply_gradients(zip(grads+[grad_tf],PINN.trainable_variables+[theta_fiber]))

  loss_value_val, _, _,_,_ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)


  if ((iter+1) % 100 == 0):
    print('iter =  '+str(iter+1))
    #loss_value_np=loss_value.numpy()
    #print('loss = {:.4f}'.format(loss_value_np ))
    tf.print('loss =' , loss_value)
    tf.print('loss_val_param =' , loss_value_val)

    theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
    a_ratio_res=1+8*a_ratio
    y0_res=-1.5+3*y0
    print(theta_fiber_res.numpy())
    print(a_ratio_res.numpy())
    print(y0_res.numpy())


for iter in range(800):


  loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

  param1_optimizer.apply_gradients(zip(grads+[grad_ar],PINN.trainable_variables+[a_ratio]))

  loss_value_val, _, _,_,_ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)


  if ((iter+1) % 100 == 0):
    print('iter =  '+str(iter+1))
    tf.print('loss =' , loss_value)
    tf.print('loss_val_param =' , loss_value_val)

    theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
    a_ratio_res=1+8*a_ratio
    y0_res=-1.5+3*y0
    print(theta_fiber_res.numpy())
    print(a_ratio_res.numpy())
    print(y0_res.numpy())


for iter in range(800):


  loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

  param2_optimizer.apply_gradients(zip(grads+[grad_y0],PINN.trainable_variables+[y0]))

  loss_value_val, _, _,_,_ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)


  if ((iter+1) % 100 == 0):
    print('iter =  '+str(iter+1))
    #loss_value_np=loss_value.numpy()
    #print('loss = {:.4f}'.format(loss_value_np ))
    tf.print('loss =' , loss_value)
    tf.print('loss_val_param =' , loss_value_val)

    theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
    a_ratio_res=1+8*a_ratio
    y0_res=-1.5+3*y0
    print(theta_fiber_res.numpy())
    print(a_ratio_res.numpy())
    print(y0_res.numpy())

patience = float('inf')
patience_lr= float('inf')
min_delta = 1e-9
best_val_loss = float('inf')
wait = 0
count = 0


param0_optimizer.learning_rate=0.003
param1_optimizer.learning_rate=0.003
param2_optimizer.learning_rate=0.003

#tf_optimizer.learning_rate=0.003

for iter in range(5000):


  loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

  #tf_optimizer.apply_gradients(zip(grads+[grad_tf, grad_ar, grad_y0],PINN.trainable_variables+[theta_fiber,a_ratio,y0]))

  param0_optimizer.apply_gradients(zip(grads+[grad_tf],PINN.trainable_variables+[theta_fiber]))
  param1_optimizer.apply_gradients(zip(grads+[grad_ar],PINN.trainable_variables+[a_ratio]))
  param2_optimizer.apply_gradients(zip(grads+[grad_y0],PINN.trainable_variables+[y0]))


  loss_value_val, _, _,_,_ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)


  best_weigths = None
  best_params = None

  # Early stopping
  if loss_value_val < best_val_loss - min_delta:
      best_val_loss = loss_value_val
      wait = 0
      count = 0
      best_weights = PINN.get_weights()
      best_params = theta_fiber.numpy()
  else:
      wait += 1
      count += 1

      if count >= patience_lr:
         tf_optimizer.learning_rate = tf_optimizer.learning_rate * 0.9
         count = 0

      if wait >= patience:
          print('Early stopping at epoch', iter + 1)
          break


  if ((iter+1) % 100 == 0):
    print('iter =  '+str(iter+1))
    #loss_value_np=loss_value.numpy()
    #print('loss = {:.4f}'.format(loss_value_np ))
    tf.print('loss =' , loss_value)
    tf.print('loss_val_param =' , loss_value_val)

    theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
    a_ratio_res=1+8*a_ratio
    y0_res=-1.5+3*y0
    print(theta_fiber_res.numpy())
    print(a_ratio_res.numpy())
    print(y0_res.numpy())


    """
    PINN_flat = PINN(X_flat)
    mse = 0.0
    time_pred = tf.reshape(PINN_flat, (151, 151))
    time_pred = time_pred.numpy()
    xmeas=xmeas.numpy().squeeze()
    ymeas=ymeas.numpy().squeeze()
    tmeas=tmeas.numpy().squeeze()

    for k in range(20):
            i, j = np.where((X == xmeas[k]) & (Y == ymeas[k]))
            mse+= (time_pred[i, j] - tmeas[k]) ** 2

    print('mse error: %.4e' % (np.sqrt(mse/20)))

    xmeas = tf.constant(xmeas.reshape(20, 1), dtype=tf.float64)
    ymeas = tf.constant(ymeas.reshape(20, 1), dtype=tf.float64)
    tmeas = tf.constant(tmeas.reshape(20, 1), dtype=tf.float64)
    """
  #PINN.set_weights(best_weights)


iter =  100
loss = [[0.00033703637150137396]]
loss_val_param = [[0.00027122341508657537]]
[0.]
[5.]
[0.708]
iter =  200
loss = [[3.2281980658142159e-05]]
loss_val_param = [[2.1311544173359117e-05]]
[0.]
[5.]
[0.708]
iter =  300
loss = [[2.9550567805516069e-05]]
loss_val_param = [[1.6938321101401347e-05]]
[0.]
[5.]
[0.708]
iter =  400
loss = [[9.6367240689668348e-06]]
loss_val_param = [[2.3473020225502334e-05]]
[0.]
[5.]
[0.708]
iter =  500
loss = [[6.5999510242052426e-06]]
loss_val_param = [[1.1923471781132276e-05]]
[0.]
[5.]
[0.708]
iter =  600
loss = [[4.93651148971412e-06]]
loss_val_param = [[1.3455852166444003e-05]]
[0.]
[5.]
[0.708]
iter =  700
loss = [[3.4931300413738847e-06]]
loss_val_param = [[1.5529365621550857e-05]]
[0.]
[5.]
[0.708]
iter =  800
loss = [[2.8493617791355919e-06]]
loss_val_param = [[1.2806255882895748e-05]]
[0.]
[5.]
[0.708]
iter =  100
loss = [[2.3118410386760886e-06]]
loss_val_param = [[1.4621762186044933e-05]]
[-0.0275755]
[5.]
[0.708]
iter =  200
loss = [[1

In [ ]:
m=20
A = np.hstack(   ( np.reshape(xmeas, (m, 1)) , np.reshape(ymeas, (m, 1)) ) )
A = np.hstack(   ( A, np.ones((m, 1)) ))
b = np.reshape(tmeas, (m, 1))
theta = np.dot(np.dot( np.linalg.pinv(np.dot(A.transpose(), A)), A.transpose()), b)
#alpha = np.arccos(-1/np.sqrt(theta[0]**2+theta[1]**2+1))
( np.arcsin(theta[1]))
#alpha =np.rad2deg( np.arctan(theta[1]/theta[0]) )
#print("angle = %.2f\n" % (alpha[0]))


In [ ]:
mse=0.0
PINN_flat = PINN(X_flat)
time_pred_net = tf.reshape(PINN_flat, (151, 151))

for i in range(151):
  for j in range(151):
    mse += (time_pred[i, j] - time_pred_net[i,j]) ** 2

mse_sq= np.sqrt(mse/22801)

print(np.mean(mse_sq))

In [ ]:
plt.figure()
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(X,Y,time_pred, cmap = 'viridis')
ax.axes.set_zlim3d(bottom=0, top=np.max(time_pred))
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

In [ ]:
plt.figure()
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(X,Y,time_pred_net, cmap = 'viridis')
ax.axes.set_zlim3d(bottom=0, top=np.max(time_pred))
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

In [ ]:
np.where(time_pred_net==np.min(time_pred_net))

In [ ]:
Y[118,]

In [ ]:
#Display results

N_h = 150
X_plot, Y_plot = np.meshgrid(np.linspace(-1.5,1.5,1501), np.linspace(-1.5,1.5,1501))

fig = plt.figure(figsize=(16,9),dpi=150)
#fig = plt.figure()
#fig.subplots_adjust(wspace=0.3)
plt.style.use('default')
ax = fig.add_subplot(1,3,1)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, u_true)
plt.scatter(xmeas,ymeas,marker='x',s=3)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
#ax.set_yticklabels(['-1.0','-0.6','-0.2','0.2','0.6','1.0'])
ax.set_title('Exact Solution',fontsize=16)

ax = fig.add_subplot(1,3,2)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, np.reshape(PINN_flat,(N_h,N_h)))
plt.scatter(xmeas,ymeas,marker='x',s=3)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
ax.set_title('PINN Prediction'.format(err),fontsize=16)

ax = fig.add_subplot(1,3,3)
ax.set_aspect(1)
im = plt.pcolor(X_plot, Y_plot, np.abs( np.reshape(PINN_flat,(N_h,N_h)) -u_true ) )
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im,cax=cax)
ax.set_title('L2 error = {:.4f}'.format(err),fontsize=16)